In [1]:
#Main Function
import requests
from bs4 import BeautifulSoup
import time
import threading
import urllib.request #Save Image
import webbrowser #멮쥐
#File
from datetime import datetime
import os
from PIL import ImageTk
import PIL.Image #ImageTk와 네임스페이스 충돌하여 따로 설정
from tkinter import filedialog
#UI
from tkinter import *
from tkinter import ttk
import tkinter.messagebox

In [2]:
#메인 로직
def TempCode_InputCharacterInfo():
    GUI.combobox_server.set("스카니아")
    GUI.combobox_job_category.set("[모전]")
    GUI.combobox_job.set("히어로")
    GUI.entry_level_min.insert(0, '1')
    GUI.entry_level_max.insert(0, '30')
    GUI.chbtn_self_setpage.select()
    
    GUI.entry_union_min.insert(0, '8500')
    GUI.entry_union_max.insert(0, '30000')
    GUI.entry_achieve_min.insert(0, '0')
    GUI.entry_achieve_max.insert(0, '30000')
    GUI.entry_ingido_min.insert(0, '-99999')
    GUI.entry_ingido_max.insert(0, '99999')    
    
def ReturnHTML(url):
    #Set User-Agent(Chrome)
    session = requests.Session()
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }
    html = session.get(url, headers=headers).content
    return BeautifulSoup(html, "html.parser")  

class File():
    
    def MakeDirectory(dir_name):
        os.makedirs(dir_name, exist_ok=True)
    
    def SetupFile(file_route):
        file = open(file_route,'w', encoding = "utf-8")
        file.close()
        
    def CsvFileSelect(): 
        #파일 선택(csv파일 우선)
        GUI.win.file = filedialog.askopenfile(
            title='파일 선택창',   
            filetypes=(('csv files', '*.csv'), ('all files', '*.*')))   
        
    def GetCsvText():
        file = open(GUI.win.file.name,'r')
        info_list = file.readlines()
        file.close()
        return info_list
    
def IsExistError():
    exist_error = False
    error_num = 0
    error_message = ""
    
    if GUI.radio_variety.get() == 1:
        if GUI.combobox_server.get() == '서버 선택':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 서버를 선택해주세요.\n"

        if GUI.combobox_job_category.get() == '직업군 선택':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 직업군을 선택해주세요.\n"

        if GUI.combobox_job.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 직업을 선택해주세요.\n"

        if GUI.entry_level_min.get() == '':
            exist_error = True
            error_num += 1
            if GUI.is_checked_self_setpage.get() != True:
                error_message += str(error_num)+". 레벨 최소 범위를 입력해주세요.\n" 
            else:
                error_message += str(error_num)+". 페이지 최소 범위를 입력해주세요.\n" 

        if GUI.entry_level_max.get() == '':
            exist_error = True
            error_num += 1
            if GUI.is_checked_self_setpage.get() != True:
                error_message += str(error_num)+". 레벨 최대 범위를 입력해주세요.\n" 
            else:
                error_message += str(error_num)+". 페이지 최대 범위를 입력해주세요.\n" 

        try:
            if int(GUI.entry_level_min.get()) > int(GUI.entry_level_max.get()):
                exist_error = True
                error_num += 1
                if GUI.is_checked_self_setpage.get() != True:
                    error_message += str(error_num)+". 레벨 최대 수치가 최소보다 작습니다.\n" 
                else:
                    error_message += str(error_num)+". 페이지 최대 수치가 최소보다 작습니다.\n" 
        except:
            if GUI.entry_level_min.get() != '' or GUI.entry_level_max.get() != '':
                exist_error = True
                error_num += 1
                if GUI.is_checked_self_setpage.get() != True:
                    error_message += str(error_num)+". 레벨 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n" 
                else:
                    error_message += str(error_num)+". 페이지 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n" 
        
    elif GUI.radio_variety.get() == 2:        
        if GUI.entry_union_min.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 유니온 최소 범위를 입력해주세요.\n"

        if GUI.entry_union_max.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 유니온 최대 범위를 입력해주세요.\n"

        try:
            if int(GUI.entry_union_min.get()) > int(GUI.entry_union_max.get()):
                exist_error = True
                error_num += 1
                error_message += str(error_num)+". 유니온 최대 수치가 최소보다 작습니다.\n"  
        except:
            if GUI.entry_union_min.get() != '' or GUI.entry_union_max.get() != '':
                exist_error = True
                error_num += 1
                error_message += str(error_num)+". 유니온 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n"        

        if GUI.entry_achieve_min.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 업적 최소 범위를 입력해주세요.\n"

        if GUI.entry_achieve_max.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 업적 최대 범위를 입력해주세요.\n"      

        try:
            if int(GUI.entry_achieve_min.get()) > int(GUI.entry_achieve_max.get()):
                exist_error = True
                error_num += 1
                error_message += str(error_num)+". 업적 최대 수치가 최소보다 작습니다.\n"  
        except:
            if GUI.entry_achieve_min.get() != '' or GUI.entry_achieve_max.get() != '':
                exist_error = True
                error_num += 1
                error_message += str(error_num)+". 업적 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n" 

        if GUI.entry_ingido_min.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 인기도 최소 범위를 입력해주세요.\n"

        if GUI.entry_ingido_max.get() == '':
            exist_error = True
            error_num += 1
            error_message += str(error_num)+". 인기도 최대 범위를 입력해주세요.\n"     

        try:
            if int(GUI.entry_ingido_min.get()) > int(GUI.entry_ingido_max.get()):
                exist_error = True
                error_num += 1
                error_message += str(error_num)+". 인기도 최대 수치가 최소보다 작습니다.\n"  
        except:
            if GUI.entry_ingido_min.get() != '' or GUI.entry_ingido_max.get() != '':
                exist_error = True
                error_num += 1
                error_message += str(error_num)+". 인기도 범위 설정에서 숫자가 아닌 것이 입력되었습니다.\n"          

    if exist_error == True:
        tkinter.messagebox.showerror("Error Message", "오류를 발견하였습니다!\n\n" + error_message)   
        return True
    else: 
        return False 
    
def SetSearchRange():
    global g_search_range
    goal_level = int(GUI.entry_level_min.get())
    if goal_level > 280: 
        g_search_range = 3
    elif goal_level >= 270 and goal_level <= 280: 
        g_search_range = 5
    elif goal_level >= 260 and goal_level <= 269: 
        g_search_range = 15
    elif goal_level >= 250 and goal_level <= 259: 
        g_search_range = 30
    elif goal_level >= 240 and goal_level <= 249: 
        g_search_range = 60
    elif goal_level >= 230 and goal_level <= 239: 
        g_search_range = 100
    elif goal_level >= 220 and goal_level <= 229: 
        g_search_range = 150        
    elif goal_level >= 210 and goal_level <= 219: 
        g_search_range = 300
    else:
        g_search_range = 500
        
def SetServerCode():
    global g_search_range
    server = GUI.combobox_server.get()
    if server == '전체월드': 
        server_code = ''
        g_search_range = int(g_search_range*2)
    elif server == '리부트2': 
        server_code = '&w=1'
        g_search_range = int(g_search_range*0.3)
    elif server == '리부트1': 
        server_code = '&w=2'
        g_search_range = int(g_search_range*0.7)
    elif server == '오로라': 
        server_code = '&w=3'
        g_search_range = int(g_search_range*0.75)
    elif server == '레드': 
        server_code = '&w=4'
        g_search_range = int(g_search_range*0.3)
    elif server == '이노시스': 
        server_code = '&w=5'
        g_search_range = int(g_search_range*0.3)
    elif server == '유니온': 
        server_code = '&w=6'
        g_search_range = int(g_search_range*0.3)
    elif server == '스카니아': 
        server_code = '&w=7'
    elif server == '루나': 
        server_code = '&w=8'
    elif server == '제니스': 
        server_code = '&w=9'
        g_search_range = int(g_search_range*0.3)
    elif server == '크로아': 
        server_code = '&w=10'
        g_search_range = int(g_search_range*0.85)
    elif server == '베라': 
        server_code = '&w=11'
        g_search_range = int(g_search_range*0.8)
    elif server == '엘리시움': 
        server_code = '&w=12'
        g_search_range = int(g_search_range*0.9)
    elif server == '아케인': 
        server_code = '&w=13'
        g_search_range = int(g_search_range*0.2)
    elif server == '노바': 
        server_code = '&w=14'
        g_search_range = int(g_search_range*0.1)
        
    return server_code

def SetJobCode():
    global g_search_range
    job = GUI.combobox_job.get()
    if job == '히어로' : job_code = '&j=1&d=12'; g_search_range = int(g_search_range*0.8)
    elif job == '팔라딘' : job_code = '&j=1&d=22'; g_search_range = int(g_search_range*0.7)
    elif job == '다크나이트' : job_code = '&j=1&d=32'; g_search_range = int(g_search_range*0.7)
        
    elif job == '불독' : job_code = '&j=2&d=12'; g_search_range = int(g_search_range*0.8)
    elif job == '썬콜' : job_code = '&j=2&d=22'; g_search_range = int(g_search_range*0.9)
    elif job == '비숍' : job_code = '&j=2&d=32'; g_search_range = int(g_search_range*0.8)
        
    elif job == '보우마스터' : job_code = '&j=3&d=12'; g_search_range = int(g_search_range*0.5)
    elif job == '신궁' : job_code = '&j=3&d=22'; g_search_range = int(g_search_range*0.6)
    elif job == '패스파인더' : job_code = '&j=3&d=32'; g_search_range = int(g_search_range*0.9)
        
    elif job == '나이트로드' : job_code = '&j=4&d=12'; g_search_range = int(g_search_range*0.9)
    elif job == '섀도어' : job_code = '&j=4&d=22'; g_search_range = int(g_search_range*0.8)
    elif job == '듀얼블레이더' : job_code = '&j=4&d=34'; g_search_range = int(g_search_range*0.95)
        
    elif job == '바이퍼' : job_code = '&j=5&d=12'; g_search_range = int(g_search_range*0.7)
    elif job == '캡틴' : job_code = '&j=5&d=22'; g_search_range = int(g_search_range*0.5)
    elif job == '캐논슈터' : job_code = '&j=5&d=32'; g_search_range = int(g_search_range*0.6)
        
    elif job == '소울마스터' : job_code = '&j=6&d=11'; g_search_range = int(g_search_range*0.7)
    elif job == '플레임위자드' : job_code = '&j=6&d=12'; g_search_range = int(g_search_range*0.5)
    elif job == '윈드브레이커' : job_code = '&j=6&d=13'; g_search_range = int(g_search_range*0.7)
    elif job == '나이트워커' : job_code = '&j=6&d=14'; g_search_range = int(g_search_range*0.5)    
    elif job == '스트라이커' : job_code = '&j=6&d=15'; g_search_range = int(g_search_range*0.5)  
    elif job == '미하일' : job_code = '&j=6&d=16'; g_search_range = int(g_search_range*0.4) 

    elif job == '배틀메이지' : job_code = '&j=10&d=32'; g_search_range = int(g_search_range*0.5)
    elif job == '와일드헌터' : job_code = '&j=10&d=33'; g_search_range = int(g_search_range*0.4)
    elif job == '메카닉' : job_code = '&j=10&d=35'; g_search_range = int(g_search_range*0.5)
    elif job == '데몬슬레이어' : job_code = '&j=10&d=31'; g_search_range = int(g_search_range*0.6)
    elif job == '데몬어벤져' : job_code = '&j=10&d=131'; g_search_range = int(g_search_range*0.7)
    elif job == '제논' : job_code = '&j=10&d=36'; g_search_range = int(g_search_range*0.5)
    elif job == '블래스터' : job_code = '&j=10&d=37'; g_search_range = int(g_search_range*0.5)
        
    elif job == '아란' : job_code = '&j=7'; g_search_range = int(g_search_range*0.7)
    elif job == '에반' : job_code = '&j=8'; g_search_range = int(g_search_range*0.7)
    elif job == '메르세데스' : job_code = '&j=11'  ; g_search_range = int(g_search_range*0.7)
    elif job == '팬텀' : job_code = '&j=12'; g_search_range = int(g_search_range*0.8)
    elif job == '루미너스' : job_code = '&j=13'; g_search_range = int(g_search_range*0.6)
    elif job == '은월' : job_code = '&j=18'; g_search_range = int(g_search_range*0.7)
        
    elif job == '카이저' : job_code = '&j=14'; g_search_range = int(g_search_range*0.7)
    elif job == '엔버' : job_code = '&j=15'; g_search_range = int(g_search_range*0.7)
    elif job == '카데나' : job_code = '&j=21'; g_search_range = int(g_search_range*0.6)
    elif job == '카인' : job_code = '&j=26'; g_search_range = int(g_search_range*0.6)

    elif job == '아크' : job_code = '&j=23'; g_search_range = int(g_search_range*0.8)
    elif job == '일리움' : job_code = '&j=22'; g_search_range = int(g_search_range*0.4)
    elif job == '아델' : job_code = '&j=25';

    elif job == '호영' : job_code = '&j=24'; g_search_range = int(g_search_range*0.8)
    elif job == '라라' : job_code = '&j=28'; g_search_range = int(g_search_range*0.5)

    elif job == '키네시스' : job_code = '&j=20'; g_search_range = int(g_search_range*0.6)
    elif job == '제로' : job_code = '&j=17' ; g_search_range = int(g_search_range*0.7)

    return job_code
    
def SetMapleRankURL(page, job_code, server_code):
    return "https://maplestory.nexon.com/Ranking/World/Total?page="+page+job_code+server_code 

def GetCharDefaultInfo(maple_html):
    nick_info_list = []
    info_html_list = maple_html.select('table.rank_table > tbody > tr')
    for info in info_html_list:
        nick = info.select_one('td.left > dl > dt').get_text()
        level = info.contents[5].get_text()
        ingido = info.contents[9].get_text().replace(',','')
        guild = info.contents[11].get_text()
        if guild == '' : guild = '-'
        char_img_url = info.select_one('span.char_img > img').get('src')
        nick_info_list.append([nick, level, ingido, guild, char_img_url])
    return nick_info_list
        
def GetUnionScore(nick):
    try:
        url = "https://maplestory.nexon.com/Ranking/Union?c="+nick
        html = ReturnHTML(url)
        union_score = int(html.select_one("tr.search_com_chk").contents[5].get_text().replace(',',''))
        return str(union_score)
    except: #If doen't have Union Score
        return '0'
        
def GetAchieveScore(nick):
    try:
        url = "https://maplestory.nexon.com/Ranking/Achievement?c="+nick
        html = ReturnHTML(url)
        achieve_score = int(html.select_one("tr.search_com_chk").contents[7].get_text().replace(',',''))
        return str(achieve_score)
    except: #If doen't set representative Character
        return '0'
            
def ExtractLevelTop(maple_html):
    info_html = maple_html.select_one('table.rank_table > tbody > tr')
    level = info_html.contents[5].get_text().replace('Lv.', '')
    return int(level)
        
def SetPageMinMax(server_code, job_code):
    global g_page_min, g_page_max, g_search_range
    
    #페이지 설정이 체크가 된 경우 
    if GUI.is_checked_self_setpage.get() == True:
        g_page_min = int(GUI.entry_level_min.get()) 
        g_page_max = int(GUI.entry_level_max.get()) 
        return 0
    
    #아닌 경우 자동 범위설정
    if g_search_range < 1:
        g_search_range = 1
    goal_level_min = int(GUI.entry_level_min.get())
    goal_level_max = int(GUI.entry_level_max.get())
    page_level = 999
    
    #첫 시작 지점 설정
    url = SetMapleRankURL(str(3*g_search_range), job_code, server_code)
    print(url)
    if ExtractLevelTop(ReturnHTML(url)) > goal_level_max: 
        i = 3
    else:
        i = 0
    
    #시작 페이지 설정
    while(page_level > goal_level_max):
        if g_is_run_getinfo == False:
            return 0
        url = SetMapleRankURL(str(i*g_search_range), job_code, server_code)
        page_level = ExtractLevelTop(ReturnHTML(url))
        UiText.InformationLabel(new_text = '검색 페이지 범위를 설정 중 입니다..(Lv :'+str(page_level)+')')
        print('current page : ', i*g_search_range, ' // Page Level :', page_level)
        i += 1
        time.sleep(0.9)
        
    g_page_min = (i-2)*g_search_range
    
    #끝 페이지 설정
    while(page_level >= goal_level_min):
        if g_is_run_getinfo == False:
            return 0        
        url = SetMapleRankURL(str(i*g_search_range), job_code, server_code)
        page_level = ExtractLevelTop(ReturnHTML(url))
        UiText.InformationLabel(new_text = '검색 페이지 범위를 설정 중 입니다..(Lv :'+str(page_level)+')')
        print('current page : ', i*g_search_range, ' // Page Level :', page_level)
        i += 1
        time.sleep(0.9)    
    g_page_max = (i-1)*g_search_range
    UiText.ResetStatusLabels()
    
def IsSuspect(character_info):
    union_min = int(GUI.entry_union_min.get())
    union_max = int(GUI.entry_union_max.get())
    achieve_min = int(GUI.entry_achieve_min.get())
    achieve_max = int(GUI.entry_achieve_max.get())
    ingido_min = int(GUI.entry_ingido_min.get())
    ingido_max = int(GUI.entry_ingido_max.get())
    union = character_info[2]
    achieve = character_info[3]
    ingido = character_info[4]   
    
    if GUI.is_checked_no_achieve_btn.get() == False:
        if int(union) >= union_min and int(union) <= union_max and int(achieve) >= achieve_min and int(achieve) <=achieve_max and int(ingido) >= ingido_min and int(ingido) <=ingido_max:
            return True
    else:
        if int(union) >= union_min and int(union) <= union_max and (int(achieve) >= achieve_min and int(achieve) <=achieve_max or int(achieve) == 0) and int(ingido) >= ingido_min and int(ingido) <=ingido_max:
            return True
    
def GetFirstMuLungDojoDate(nick):
    try:
        url = 'https://maple.gg/u/'+nick
        html = ReturnHTML(url)
        mulung_list = html.select('#dohangHistoryTbody > tr')
        if mulung_list != []:
            first_mulung_date = mulung_list[len(mulung_list)-1].contents[1].get_text().strip().replace('\n','')
            return first_mulung_date
        else:
            return None
    except:
        #GG사이트 구조 변경으로 인한 오류 방지(try - except)
        return None
            
class Result():
    def SaveResultFile(character_info_list):
        if character_info_list == []:
            tkinter.messagebox.showinfo("Message", "검색 된 캐릭터가 없습니다.")
            return 0
        #Change Ui
        UiText.ResetStatusLabels()
        UiText.InformationLabel(new_text = '결과를 저장하는 중 입니다..') 
        
        #Make file and input data
        dir_name = datetime.today().strftime("%Y-%m-%d-%H%M%S")
        File.MakeDirectory(dir_name) 
        File.SetupFile(dir_name+'/결과.csv')
        csv_file = open(dir_name+'/결과.csv', "w")
        csv_file.write('닉네임'+','+'레벨'+','+'유니온'+','+'업적'+','+'인기도'+','+'길드'+'\n')
        for i, character_info in enumerate(character_info_list, 1):        
            nick = character_info[0]
            level = character_info[1]
            union = character_info[2]
            achieve = character_info[3]
            ingido = character_info[4]
            guild = character_info[5]
            img_url = character_info[6]
            #Csv파일에 정보 입력
            csv_file.write(nick+','+level+','+union+','+achieve+','+ingido+','+guild+','*20+img_url+'\n')  
            #진행상황 표시
            percent = round((i/len(character_info_list))*100,1)            
            UiText.InformationLabel(new_text = '결과를 저장하는 중 입니다..[ '+str(percent)+'% ]')
            
        #End Message    
        tkinter.messagebox.showinfo("Message", "결과가 '"+dir_name+"' 폴더에 저장되었습니다.")
        UiText.ResetStatusLabels()
        csv_file.close()

    def ShowResultWindow(character_info_list):
        def GoToMapleGG(event):
            nick = character_info_list[character_page][0]
            url = 'https://maple.gg/u/'+nick
            webbrowser.open(url)
            
        def BeforeInfo():
            nonlocal character_page
            if character_page <= 0: #First Page -> Last Page
                character_page = character_amount - 1 
                ShowCharacterInfo()
            else:
                character_page -= 1
                ShowCharacterInfo()

        def NextInfo():
            nonlocal character_page
            if character_page+1 >= character_amount: #Last Page -> First Page
                character_page = 0
                ShowCharacterInfo()
            else:
                character_page += 1
                ShowCharacterInfo()
                
        def ShowCharacterInfo():
            start_time = time.time()
            nick = character_info_list[character_page][0]
            level = character_info_list[character_page][1]
            union = character_info_list[character_page][2]
            achieve = character_info_list[character_page][3]
            ingido = character_info_list[character_page][4]
            guild = character_info_list[character_page][5]
            img_url = character_info_list[character_page][6]
            mulung_date = character_info_list[character_page][7]
            if union == '0': union = '-'
            if achieve == '0': achieve = '-'
                
            #Get img raw
            character_img = ImageTk.PhotoImage(PIL.Image.open(requests.get(img_url, stream=True).raw))
            
            #Info to Toplevel
            result_window.title("결과["+str(character_page+1)+'/'+str(character_amount)+']')
            label_character_img.config(image = character_img)
            label_character_img.image = character_img # 파이썬 가비지 컬렉션의 이미지 삭제 방지        
            label_nick.config(text = nick+'(Lv.'+level+')')
            label_union.config(text = '유니온 : '+union)
            label_achieve.config(text = '업적 : '+achieve)
            label_ingido.config(text = '인기도 : '+ingido)
            label_guild.config(text = '길드 : '+guild)  
            
            if mulung_date != None:
                date = mulung_date.replace('년', '').replace('월', '').replace('일', '').split(' ')
                time1 = datetime(int(date[0]), int(date[1]), int(date[2]))
                time2 = datetime.now()
                gap = str(time2-time1).split(' ')[0]
                if int(gap) > 60:
                    label_mulung.config(text = '60일 내 닉변 가능성이 없습니다')
                else : 
                    label_mulung.config(text = '')
            else:
                label_mulung.config(text = '')    
                
        #Toplevel Ui
        character_page = 0
        character_amount = len(character_info_list)
        result_window = Toplevel()
        result_window.title("결과")
        result_window.geometry('230x345')
        result_window.option_add('*Font', '맑은고딕 9')
        result_window.resizable(False, False)
        label_character_img = Label(result_window)
        label_character_img.bind('<Button-1>', GoToMapleGG)
        label_img_caption = Label(result_window, text = '이미지 클릭 시 메이플GG로 이동합니다!', font = '맑은고딕 8')
        label_nick = Label(result_window)
        label_union = Label(result_window)
        label_achieve = Label(result_window)
        label_ingido = Label(result_window)
        label_guild = Label(result_window)    
        label_mulung = Label(result_window, foreground='blue')  
        BeforeBtn = Button(result_window, width = 10, text = '이전', command = BeforeInfo)
        NextBtn = Button(result_window, width = 10, text = '다음', command = NextInfo)
        
        label_img_caption.pack()
        label_character_img.pack()
        label_nick.pack()
        label_union.pack()
        label_achieve.pack()
        label_ingido.pack()
        label_guild.pack()
        label_mulung.pack()
        BeforeBtn.place(x = 5, y = 317)
        NextBtn.place(x = 145, y = 317)
        ShowCharacterInfo()
        
#Global Var
g_is_run_getinfo = False
g_is_run_filter = False
g_search_range = 0
g_page_min = 0
g_page_max = 0

In [3]:
#메인 로직 실행 부분 
def ActivateMainFunction_Thread():
    if GUI.radio_variety.get() == 1:
        t1=threading.Thread(target=ActivateGetInfoMode)
        t1.setDaemon(True)
        t1.deamon = True
        t1.start()
    else:
        t1=threading.Thread(target=ActivateFilterMode)
        t1.setDaemon(True)
        t1.deamon = True
        t1.start()             

def DeActivateGetInfoMode():
    global g_is_run_getinfo
    g_is_run_getinfo = False
    UiText.InfoButton(new_text = 'Activate')
    UiText.ResetStatusLabels()
    UiText.EnabledModeUi()
    return 0

def DeActivateFilterMode():
    global g_is_run_filter
    g_is_run_filter = False
    UiText.FilterButton(new_text = 'Activate')
    UiText.ResetStatusLabels()
    UiText.EnabledModeUi()
    return 0

def ActivateGetInfoMode():
    #If Click DeActivateBtn
    global g_is_run_getinfo
    if g_is_run_getinfo == True :
        DeActivateGetInfoMode()
        tkinter.messagebox.showinfo("Message", "사용자 요청에 따라 작동을 중지합니다.")
        return 0
    else : 
        g_is_run_getinfo = True
        UiText.InfoButton(new_text = 'DeActivate')
        
#     TempCode_InputCharacterInfo()    
    
    #Check Error
    if IsExistError() == True:
        DeActivateGetInfoMode()
        tkinter.messagebox.showerror("Error Message", "오류를 발견하여 작동을 중지합니다.\n수정 후 재시도 바랍니다.")
        return 0 
    
    #Set Var
    global g_page_min, g_page_max
    SetSearchRange()
    server_code = SetServerCode()
    job_code = SetJobCode()
    SetPageMinMax(server_code, job_code)
    goal_level_min = int(GUI.entry_level_min.get())
    goal_level_max = int(GUI.entry_level_max.get())
    character_info_list = []
    skip_stack = 0
    time_sum = 0
    
    #Set Ui
    UiText.ResetStatusLabels()
    UiText.DisabledModeUi()
    
    for present_page in range(g_page_min, g_page_max+1):
        start_time = time.time()
        #홈페이지로부터 페이지당 10명의 닉네임과 인기도, 길드, 캐릭터 이미지 url 추출
        maple_url = SetMapleRankURL(str(present_page), server_code, job_code)
        maple_html = ReturnHTML(maple_url)
        nick_info_list = GetCharDefaultInfo(maple_html)
        
        #설정한 레벨 범위 초과된 페이지 스킵
        if GUI.is_checked_self_setpage.get() == False and g_is_run_getinfo != False:  
            top_level = int(nick_info_list[0][1].replace('Lv.', ''))
            bottom_level =  int(nick_info_list[9][1].replace('Lv.', ''))
            if top_level > goal_level_max and bottom_level > goal_level_max:
                time.sleep(0.9)
                g_page_min = present_page+1 
                skip_stack += 1
                UiText.CalculateProgressPercent(10, present_page, g_page_min-skip_stack, g_page_max)
                UiText.CurrentPage(current_page = present_page-g_page_min+skip_stack+1, end_page = g_page_max-g_page_min+skip_stack+1)
                continue
            elif top_level < goal_level_min and bottom_level < goal_level_min:
                g_page_max = present_page-1
                UiText.ProgressText(new_text = '100%')
                tkinter.messagebox.showinfo("Message", "설정한 레벨 구간을 넘긴 페이지를 모두 스킵 하였습니다.")
                break
                
        #(2). 10명의 캐릭터 각각의 정보를 입력 조건과 비교
        for i in range(10): 
            #사용자가 Deactivate를 누르면 동작 중지
            if g_is_run_getinfo == False:
                DeActivateGetInfoMode()
                time.sleep(0.7)
                if character_info_list != [] and tkinter.messagebox.askquestion ('Message','결과를 저장하시겠습니까?') == 'yes':
                    Result.SaveResultFile(character_info_list)
                return 0  
            #기본 정보 설정
            nick = nick_info_list[i][0]
            level = nick_info_list[i][1].replace('Lv.', '')
            ingido = nick_info_list[i][2]
            guild = nick_info_list[i][3]
            char_img_url = nick_info_list[i][4]
            #해당 캐릭터 레벨이 지정한 범위 레벨보다 작거나 크면 스킵()
            if GUI.is_checked_self_setpage.get() == False:
                if int(level) < goal_level_min or int(level) > goal_level_max:
                    print('스킵 : ', level)
                    time.sleep(0.09) 
                    continue
            #캐릭터 정보를 리스트에 저장
            character_info_list.append([nick, level, GetUnionScore(nick), GetAchieveScore(nick), ingido, guild, char_img_url])
            #진행률 % 표시
            UiText.CalculateProgressPercent(i+1, present_page, g_page_min-skip_stack, g_page_max)
            time.sleep(0.09)  
        #현재 진행 페이지 수와 예상 시간 출력
        UiText.CurrentPage(current_page = present_page-g_page_min+skip_stack+1, end_page = g_page_max-g_page_min+skip_stack+1)
        time_sum = time_sum + time.time() - start_time
        UiText.CalculateExpectedTime(time_sum, present_page, g_page_min, g_page_max)
        
    #Result
    UiText.ResetStatusLabels()
    Result.SaveResultFile(character_info_list)
    DeActivateGetInfoMode()
            
def ActivateFilterMode():
    global g_is_run_filter
    if g_is_run_filter == True:
        DeActivateFilterMode()
        tkinter.messagebox.showinfo("Message", "사용자 요청에 따라 작동을 중지합니다.")
        return 0
    else : 
        g_is_run_filter = True
        UiText.FilterButton(new_text = 'DeActivate')
        
    #테스트코드
    #TempCode_InputCharacterInfo()
    
    #Check Error
    if IsExistError() == True:
        tkinter.messagebox.showerror("Error Message", "오류를 발견하여 작동을 중지합니다.\n수정 후 재시도 바랍니다.")
        return 0 
    
    #Load Csv File
    tkinter.messagebox.showinfo("Message", "캐릭터 정보가 저장된 Csv 파일을 선택해주세요 :D\n(해당 프로그램이 만든 엑셀 파일만 넣어야합니다!)")
    try:
        File.CsvFileSelect()
        character_info_list = File.GetCsvText()
    except:
        tkinter.messagebox.showerror("Error Message", "파일 입력이 안 됐거나, 인식할 수 없는 Csv 파일입니다.")        
        return 0

    #Csv File Check
    if character_info_list[0].strip() != "닉네임,레벨,유니온,업적,인기도,길드":
        tkinter.messagebox.showerror("Error Message", "인식할 수 없는 Csv 파일입니다.\n(해당 프로그램에서 제작된 것인지 확인해주세요.)")        
        return 0
    
    #Set Var
    suspect_info_list = []
    suspect_amount = 0
    total_info_amount = len(character_info_list)
    
    #Set Ui
    UiText.DisabledModeUi()
    
    for i in range(1, total_info_amount):
        #사용자 요청 시 기존에 돌아가는 것 종료
        if g_is_run_filter == False: 
            DeActivateFilterMode()
            return 0
        character_info = character_info_list[i].strip().split(',')
        #진행률 표시
        UiText.CharacterFilterProgress(new_text='['+str(i)+'/'+str(total_info_amount-1)+']')
        #조건 비교
        if IsSuspect(character_info) == True:
            suspect_amount += 1
            nick = character_info[0]
            level = character_info[1]
            union = character_info[2]
            achieve = character_info[3]
            ingido = character_info[4]
            guild = character_info[5]
            img_url = character_info[25]
            if suspect_amount <= 50: 
                mulung_date = GetFirstMuLungDojoDate(nick)
            else : 
                mulung_date = None
            suspect_info_list.append( (nick, level, union, achieve, ingido, guild, img_url, mulung_date) ) 
        else:
            continue
                        
    #Result
    DeActivateFilterMode()
    if suspect_amount != 0:
        if suspect_amount >= 50:
            tkinter.messagebox.showwarning("Message", "조건 일치자가 50명을 초과한 경우 앞에서 50명까지만 닉변 가능성 유무가 출력됩니다.")         
        Result.ShowResultWindow(suspect_info_list)
    else:
        tkinter.messagebox.showinfo("Message", "조건에 일치하는 캐릭터가 없습니다.") 

In [4]:
#UI 관련 기능
class UiText():
    def ResetStatusLabels():
        GUI.label_information.config(text ='')
        GUI.label_progress.config(text ='')
        GUI.label_page.config(text ='')
        GUI.label_expect_time.config(text ='')
                
    def InfoButton(new_text):
        GUI.button_getinfo_mode.config(text = new_text)
        
    def FilterButton(new_text):
        GUI.button_filter_mode.config(text = new_text)
        
    def ProgressText(new_text):
        GUI.label_progress.config(text = new_text)
        
    def CurrentPage(current_page, end_page):
        GUI.label_page.config(text = '['+ str(current_page)+'/'+ str(end_page) +']')
        
    def InformationLabel(new_text):
        GUI.label_information.config(text = new_text)        
        
    def CharacterFilterProgress(new_text):
        GUI.label_charater_progress.config(text = new_text)   
        
    def CalculateProgressPercent(char_sequence, present_page, page_min, page_max): #char_sequence == 페이지 내 10명의 캐릭터 중 어디에 있는지 표현한 변수명 
        progress_total = (page_max-page_min+1)*10
        progress_present = (present_page-page_min)*10+(char_sequence)
        percent = round((progress_present/progress_total)*100,1)
        GUI.label_progress.config(text = ( str(percent)+'%' ))        
        
    def CalculateExpectedTime(time_sum, present_page, page_min, page_max):
        #기대 완료 시간 = (시간 합계/i) * 남은 페이지 
        expect_time = (time_sum/(present_page-page_min+1)) * (page_max-present_page)
        if expect_time <= 0:
            GUI.label_expect_time.config(text = '')
        else:
            GUI.label_expect_time.config(text = '예상 시간 : '+str(round(expect_time,2))+ '초')
        
    def ClearEntryText(*clear_entrys):
        for clear_entry in clear_entrys:
            if clear_entry == 'union':
                GUI.entry_union_min.delete(0, len(GUI.entry_union_min.get()))
                GUI.entry_union_max.delete(0, len(entry_union_max.get()))
            if clear_entry == 'achieve':    
                GUI.entry_achieve_min.delete(0, len(GUI.entry_achieve_min.get()))
                GUI.entry_achieve_max.delete(0, len(GUI.entry_achieve_max.get()))
            if clear_entry == 'ingido':
                GUI.entry_ingido_min.delete(0, len(GUI.entry_ingido_min.get()))
                GUI.entry_ingido_max.delete(0, len(GUI.entry_ingido_max.get()))
            if clear_entry == 'level' or clear_entry == 'page':
                GUI.entry_level_min.delete(0, len(GUI.entry_level_min.get()))
                GUI.entry_level_max.delete(0, len(GUI.entry_level_max.get()))
                
    def DisabledModeUi():
        if GUI.radio_variety.get() == 1:
            GUI.entry_level_min["state"] = 'disabled'
            GUI.entry_level_max["state"] = 'disabled'
            GUI.chbtn_self_setpage["state"] = 'disabled'
        else:
            GUI.entry_union_min["state"] = 'disabled'
            GUI.entry_union_max["state"] = 'disabled'
            GUI.entry_achieve_min["state"] = 'disabled'
            GUI.entry_achieve_max["state"] = 'disabled'
            GUI.entry_ingido_min["state"] = 'disabled'
            GUI.entry_ingido_max["state"] = 'disabled'
            GUI.chbtn_include_no_achieve["state"] = 'disabled'

    def EnabledModeUi():
        if GUI.radio_variety.get() == 1:
            GUI.entry_level_min["state"] = 'normal'
            GUI.entry_level_max["state"] = 'normal'
            GUI.chbtn_self_setpage["state"] = 'normal'
        else:
            GUI.entry_union_min["state"] = 'normal'
            GUI.entry_union_max["state"] = 'normal'
            GUI.entry_achieve_min["state"] = 'normal'
            GUI.entry_achieve_max["state"] = 'normal'
            GUI.entry_ingido_min["state"] = 'normal'
            GUI.entry_ingido_max["state"] = 'normal'
            GUI.chbtn_include_no_achieve["state"] = 'normal'
            

def ShowJobList(event):
    job_category = GUI.combobox_job_category.get()
    GUI.combobox_job.set('')
    
    if job_category == '[모전]' : job_list = ['히어로', '팔라딘', '다크나이트']
    elif job_category == '[모법]' : job_list = ['불독', '썬콜', '비숍']
    elif job_category == '[모궁]' : job_list = ['보우마스터', '신궁', '패스파인더']
    elif job_category == '[모도]' : job_list = ['나이트로드', '섀도어', '듀얼블레이더']
    elif job_category == '[모해]' : job_list = ['바이퍼', '캡틴', '캐논슈터']    
    elif job_category == '[시그너스]' : job_list = ['소울마스터', '플레임위자드', '윈드브레이커', '나이트워커', '스트라이커', '미하일'] 
    elif job_category == '[레지스탕스]' : job_list = ['배틀메이지', '와일드헌터', '메카닉','데몬슬레이어', '데몬어벤져', '제논', '블래스터'] 
    elif job_category == '[영웅]': job_list = ['아란', '에반', '메르세데스', '팬텀', '루미너스', '은월'] 
    elif job_category == '[노바]': job_list = ['카이저', '엔버', '카데나', '카인'] 
    elif job_category == '[레프]': job_list = ['아크', '일리움', '아델'] 
    elif job_category == '[아니마]': job_list = ['호영', '라라']     
    elif job_category == '[혼밥족]': job_list = ['키네시스', '제로'] 
    
    GUI.combobox_job["state"] = "readonly"
    GUI.combobox_job["values"] = job_list
        
def EnableSelfSetPage():
    if GUI.is_checked_self_setpage.get() == True:
        tkinter.messagebox.showinfo("Message", "레벨 범위 설정이 페이지 범위 설정으로 변경됩니다.\n(홈페이지에서 검색할 페이지의 범위를 입력해주세요)") 
        UiText.ClearEntryText('level')
        GUI.label_level.config(text = '[페이지 범위]')
        GUI.label_level.config(foreground = 'blue')
        GUI.label_level.place(x = 93, y = 210)
    else:
        GUI.label_level.config(text = '[레벨 범위]')
        GUI.label_level.config(foreground = 'black')
        GUI.label_level.place(x = 100, y = 210)        
        UiText.ClearEntryText('level')
        
def AchieveCheckBtnMessage():
    if GUI.is_checked_no_achieve_btn.get() == True:
        tkinter.messagebox.showinfo("Message", "기능이 활성화되어 업적이 없는 캐릭터도 검색합니다.") 

In [6]:
#UI
class UiMain():
    def __init__(self):
        self.win = Tk()
        self.win.title('추노')    
        self.win.geometry('430x310')
        self.win.option_add('*Font', '맑은고딕 10')
        self.win.resizable(False, False)
        
        #Mode Select Ui[Radio Button]
        self.radio_variety = tkinter.IntVar()
        self.mode_select_1 = Radiobutton(self.win, text="캐릭터 정보 수집 모드", font = '맑은고딕 8', value=1, variable=self.radio_variety, command = lambda:UiMain.ShowGetInfoModeUI(self))
        self.mode_select_1.select()
        self.mode_select_2 = Radiobutton(self.win, text="정보 필터링 모드", font = '맑은고딕 8', value=2, variable=self.radio_variety, command = lambda:UiMain.ShowFilterModeUI(self)) 
        self.mode_select_1.place(x = 70, y = 90)
        self.mode_select_2.place(x = 250, y = 90)
        
        #Default Ui
        UiMain.MenuBar(self)
        UiMain.MainImage(self)
        UiMain.MakeGetInfoModeUi(self)
        UiMain.MakeFilterModeUI(self)
        UiMain.ShowGetInfoModeUI(self)
        
    def MenuBar(self):
        def GoToMyBlog():
            webbrowser.open("https://blog.naver.com/PostList.naver?blogId=wkrnjs456&categoryNo=8&skinType=&skinId=&from=menu")
            
        def GoToMyGithub():
            webbrowser.open("https://github.com/JaGwonLim")
        menubar = Menu(self.win)
        menu_1 = Menu(menubar, tearoff=0)
        menu_1.add_command(label="Help", command = GoToMyBlog)
        menu_1.add_command(label="Github", command = GoToMyGithub)
        menubar.add_cascade(label="Option", menu=menu_1)
        self.win.config(menu=menubar)
        
    def MainImage(self):
        try:
            self.label_image = Label(self.win)
            main_img_url = "https://postfiles.pstatic.net/MjAyMjAxMjZfNzAg/MDAxNjQzMTc0NjkzNTYw.2DrFjsCBdwiB8m-HWXkHSPp8qKvjtMISOFbqgw9egaAg.gO_uTBQ7hCS_8k8INmH8x_awRzGyEpZq57atEpw4b5Mg.PNG.wkrnjs456/%EC%B6%94%EB%85%B8-%EB%B0%98.png?type=w966"
            main_img = ImageTk.PhotoImage(PIL.Image.open(requests.get(main_img_url, stream=True).raw))
            self.label_image.config(image = main_img)
            self.label_image.image = main_img # 파이썬 가비지 컬렉션의 이미지 삭제 방지
            self.label_image.pack()
        except:
            #네이버 구조 변경/사진 삭제 등으로 인한 오류 방지
            pass
    def MakeGetInfoModeUi(self):
        #[Set Server]
        self.label_server = Label(self.win, text = '[서버 설정]')

        self.server_list =['전체월드', '루나', '스카니아', '엘리시움', '크로아', '베라', 
                      '오로라', '레드', '제니스', '이노시스', '유니온', 
                      '아케인', '노바', '리부트1', '리부트2'] 

        self.combobox_server = ttk.Combobox(self.win, 
                                       width = 10, 
                                       values = self.server_list, 
                                       state="readonly")
        self.combobox_server.set("서버 선택")
        
        #[Set Job]
        self.label_job = Label(self.win, text = '[직업 설정]')

        self.job_category =['[모전]', '[모법]', '[모궁]', '[모도]', '[모해]',
                       '[시그너스]',
                       '[영웅]',
                       '[레지스탕스]',
                       '[노바]',
                       '[레프]',
                       '[아니마]',
                       '[혼밥족]'
                       ] 
        self.combobox_job_category = ttk.Combobox(self.win, 
                                             width = 10, 
                                             values = self.job_category, 
                                             state="readonly")
        #job_category Choice Event
        self.combobox_job_category.bind("<<ComboboxSelected>>", ShowJobList)
        #job_category Text
        self.combobox_job_category.set("직업군 선택")
        #job 
        self.combobox_job = ttk.Combobox(self.win, width = 10, state="disabled")
        
        #[Set Level]
        self.label_level = Label(self.win, text = '[레벨 범위]')
        self.entry_level_min = Entry(self.win, width = 5)
        self.entry_level_max = Entry(self.win, width = 5)
        self.label_level_range = Label(self.win, text = '-')        

        #[Progress Label]
        self.label_progress = Label(self.win)
        self.label_page = Label(self.win)
        self.label_expect_time = Label(self.win)
        self.label_information = Label(self.win, font = '맑은고딕 9')   
        
        #[Option Check Button]
        self.is_checked_self_setpage = BooleanVar()
        self.chbtn_self_setpage = Checkbutton(self.win,
                            text = "검색 페이지 직접 설정",
                            variable = self.is_checked_self_setpage,
                            font = '맑은고딕 8')
        self.chbtn_self_setpage.config(command = EnableSelfSetPage)
        
        #[Activate Button]
        self.button_getinfo_mode = Button(self.win, 
                                 width = 20,
                                 height = 2,
                                 text = 'Activate',
                                 command = ActivateMainFunction_Thread) 
        
    def MakeFilterModeUI(self):
        #[Set Union Score]
        self.label_union = Label(self.win, text = '[유니온 범위]')
        self.entry_union_min = Entry(self.win, width = 5)
        self.label_union_range = Label(self.win, text = '-')
        self.entry_union_max = Entry(self.win, width = 5)     
        
        #[Set Achievement Score]
        self.label_achieve = Label(self.win, text = '[업적 범위]')
        self.entry_achieve_min = Entry(self.win, width = 5)
        self.label_achieve_range = Label(self.win, text = '-')
        self.entry_achieve_max = Entry(self.win, width = 5)     
        
        #[Set Ingido]
        self.label_ingido = Label(self.win, text = '[인기도 범위]')
        self.entry_ingido_min = Entry(self.win, width = 5)
        self.label_ingido_range = Label(self.win, text = '-')
        self.entry_ingido_max = Entry(self.win, width = 5)
        
        #[Progress Label]
        self.label_charater_progress = Label(self.win)
        
        #[Option Check Button]
        self.is_checked_no_achieve_btn = BooleanVar()
        self.chbtn_include_no_achieve = Checkbutton(self.win,
                            text = "업적이 없어도 검색",
                            variable = self.is_checked_no_achieve_btn,
                            font = '맑은고딕 8',
                            command = AchieveCheckBtnMessage)
        
        #[Activate Button]
        self.button_filter_mode = Button(self.win, 
                                 width = 20,
                                 height = 2,
                                 text = 'Activate',
                                 command = ActivateMainFunction_Thread) 

    def ShowGetInfoModeUI(self):
        #기존 Ui 가리기
        try : UiMain.HideGetInfoModeUi(self)
        except: pass         
        try: UiMain.HideFilterModeUi(self)
        except: pass 
        
        #[Set Server]
        self.label_server.place(x = 100 , y = 150)
        self.combobox_server.place(x = 200, y = 150)        

        #[Set Job]
        self.label_job.place(x = 100 , y = 180)
        self.combobox_job_category.place(x = 200, y = 180)    
        self.combobox_job.place(x = 300, y = 180)         
        
        #[Set Level]
        if self.is_checked_self_setpage.get() == False:
            self.label_level.place(x = 100, y = 210)
        else:
            self.label_level.place(x = 93, y = 210)
        self.entry_level_min.place(x = 200, y = 210)
        self.entry_level_max.place(x = 260, y = 210)
        self.label_level_range.place(x = 240, y = 210)   
        
        #[Progress Label]
        self.label_progress.place(x = 195 , y = 243)
        self.label_page.place(x = 235 , y = 243)
        self.label_expect_time.place(x = 295 , y = 243)
        self.label_information.place(x = 120, y = 243)        

        #[Activate Button]
        self.button_getinfo_mode.place(x = 120 , y = 268)    
        
        #[Option Check Button]
        self.chbtn_self_setpage.place(x = 290, y = 285)         
        
    def ShowFilterModeUI(self):
        #기존 Ui 가리기
        try : UiMain.HideFilterModeUi(self)
        except: pass 
        try : UiMain.HideGetInfoModeUi(self)
        except: pass 
        
        #[Set Union Score]
        self.label_union.place(x = 93, y = 150)
        self.entry_union_min.place(x = 200, y = 150)
        self.label_union_range.place(x = 240, y = 150)
        self.entry_union_max.place(x = 260, y = 150)

        #[Set Achievement Score]
        self.label_achieve.place(x = 100, y = 180)
        self.entry_achieve_min.place(x = 200, y = 180)
        self.entry_achieve_max.place(x = 260, y = 180)
        self.label_achieve_range.place(x = 240, y = 180) 

        #[Set Ingido]
        self.label_ingido.place(x = 93, y = 210)
        self.entry_ingido_min.place(x = 200, y = 210)
        self.entry_ingido_max.place(x = 260, y = 210)
        self.label_ingido_range.place(x = 240, y = 210)
        
        #[Progress Label]
        self.label_charater_progress.place(x = 180, y = 243)
        
        #[Activate Button]
        self.button_filter_mode.place(x = 120 , y = 268)   
        
        #[Option Check Button]
        self.chbtn_include_no_achieve.place(x = 290, y = 285)
        
    def HideGetInfoModeUi(self):
        self.label_server.place_forget()
        self.label_job.place_forget()
        self.label_level.place_forget()
        self.label_level_range.place_forget()
        
        self.label_progress.place_forget()
        self.label_page.place_forget()
        self.label_expect_time.place_forget()
        self.label_information.place_forget()
        
        self.combobox_server.place_forget()
        self.combobox_job_category.place_forget()
        self.combobox_job.place_forget()
        
        self.entry_level_min.place_forget()
        self.entry_level_max.place_forget()
        
        self.button_getinfo_mode.place_forget()
        self.chbtn_self_setpage.place_forget()
        
    def HideFilterModeUi(self):
        self.label_union.place_forget()
        self.label_union_range.place_forget()
        self.label_achieve.place_forget()
        self.label_achieve_range.place_forget()
        self.label_ingido.place_forget()
        self.label_ingido_range.place_forget()
        
        self.label_charater_progress.place_forget()
        
        self.entry_union_min.place_forget()
        self.entry_union_max.place_forget()
        self.entry_achieve_min.place_forget()
        self.entry_achieve_max.place_forget()
        self.entry_ingido_min.place_forget()
        self.entry_ingido_max.place_forget()
        
        self.button_filter_mode.place_forget()
        self.chbtn_include_no_achieve.place_forget()
                 
GUI = UiMain()
GUI.win.mainloop()